# Retrieve Dataset

This notebook is used to collect a dataset.
The dataset contains RGB and depth images of both cameras, zivid and realsense.

Following cells are used to load all required modules and classes.

In [1]:
# import dependencies
import time
from pathlib import Path
from tqdm import tqdm
# import required modules
from camera_interfaces.real_sense import RealSense
from camera_interfaces.zivid import Zivid
from dataset.dataset_interface import DatasetInterface

## Configuration
For the collection, both cameras will take images of the scene for X times.
To configure this process a bit, following parameters are useful.

In [13]:
resource_path = Path("../resources")

class Args:
    picture_amount = 100                  # how many frames to collect
    delay = 3000                         # delay between exposures in ms
args = Args()

# Connect to cameras

In [3]:
# make sure both instances are destroyed. Otherwise they are still connected to the cameras
%reset_selective -f real_sense
%reset_selective -f zivid
# instantiate and connect to cameras
real_sense = RealSense()
real_sense.connect()
print("Connected to realsense camera")

zivid = Zivid()
zivid.connect()
print("Connected to zivid camera")
zivid.configure_manual(resource_path / Path("capture_settings.yml"))
# zivid.configure_automatically()

print("Connected to both cameras")

RealSense Configuration
camera product line: D400
camera product id:   0B07
Depth Camera Intrinsics
Principal Point (ppx ppy):    635.5166625976562 359.2284851074219
Focal Length (fx fy):         639.042724609375 639.042724609375
Distortion Coeffs:            [0.0, 0.0, 0.0, 0.0, 0.0]
Distortion Model:             distortion.brown_conrady
HeightxWidth:                 720x1280
Color Camera Intrinsics
Principal Point (ppx ppy):    936.4846801757812 537.48779296875
Focal Length (fx fy):         1377.6448974609375 1375.7239990234375
Distortion Coeffs:            [0.0, 0.0, 0.0, 0.0, 0.0]
Distortion Model:             distortion.inverse_brown_conrady
HeightxWidth:                 1080x1920
Connected to realsense camera
Zivid Configuration
Color Camera Intrinsics
Principal Point (ppx ppy):    951.68 594.779
Focal Length (fx fy):         2760.12 2759.78
Distortion Coeffs:            [-2.73150e-01  3.54379e-01 -3.44441e-04  1.98413e-04 -3.22515e-01]
Connected to zivid camera
manually configur

## Collect Pictures

The `out_path` specifies the directory, the dataset files should be saved in.
Every capture will be saved to that folder with the `time.time()` as file name.
The last cell can be executed repeatingly to capture as many frames as required.
The cell can also be stopped without a problem, as it directly saves the captured frames to the file system in each step.

In [8]:
out_path = resource_path / "images/uncalibrated/dataset_1"

In [12]:
# execute this cell to generate and append pairs of images. The images won't be saved.
for i in tqdm(range(args.picture_amount)):
    file_path = out_path / str(time.time())
    rs_rgb, rs_depth = real_sense.collect_frame()
    zv_rgb, zv_depth = zivid.collect_frame()
    DatasetInterface.save(rs_rgb, rs_depth, zv_rgb, zv_depth, mask=None, file_name=file_path)

    if i + 1 == args.picture_amount:
        time.sleep(args.delay / 1000.0)

100%|██████████| 100/100 [08:44<00:00,  5.24s/it]
